https://timeseriesreasoning.com/contents/generalized-poisson-regression-model/

In [1]:
# DOWNLOADING DATA

import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np

tomato_discount = pd.read_csv('Dataframes/tomato_discount.csv')
tomato_12 = pd.read_csv('Dataframes/tomato_12.csv') # or index_col = 0 instead of index = False in 'qjuices'

# tomato_12.drop([col for col in tomato_12.columns if
#                 'holidays_' in col or
#                 'month_' in col or
#                 'salesvolume_' in col and 'salesvolume_201676' not in col], axis = 1, inplace = True)

# tomato_12

In [2]:
# SETTING DATA TO TARGET

target_sku = '201676'

sales_names_to_shift = [col for col in tomato_12.columns if 'salesvolume_' in col and target_sku not in col]

tomato_12.loc[:, sales_names_to_shift] = tomato_12.loc[:, sales_names_to_shift].shift(1)
tomato_12 = tomato_12.iloc[1:, :]

for colname in sales_names_to_shift:
    tomato_12.rename(columns = {colname: colname + '_lag'}, inplace = True)

tomato_12.rename(columns = {'salesvolume_' + target_sku: 'target'}, inplace = True) # 'av_price_' + target_sku: 'price_of_target'
tomato_12.reset_index(drop = True, inplace = True)

print('Corr between target and it\'s price: {}'.format(round(tomato_12.corr().loc['target', 'av_price_' + target_sku], 4)))

# tomato_12

Corr between target and it's price: -0.4642


In [3]:
# ADDING more LAGS

names_to_shift = [col for col in tomato_12.columns if 'salesvolume_' in col] + \
                 [col for col in tomato_12.columns if 'av_price_' in col] + \
                 [col for col in tomato_12.columns if 'sales_week_' in col] + \
                 ['target']
#                 [col for col in tomato_12.columns if 'is_supplied_' in col] + \
for i in [1, 2, 3, 7, 14]:
    for col in names_to_shift:
        tomato_12[col + '_' + str(i)] = tomato_12[col].shift(i)

tomato_12 = tomato_12[14:]

tomato_12.reset_index(drop = True, inplace = True)

# tomato_12

In [22]:
# # GETTING RID OF DEPENDENT COLUMNS
#
# # https://stackoverflow.com/questions/44555763/is-there-a-way-to-check-for-linearly-dependent-columns-in-a-dataframe
#
# import sympy
#
# reduced_form, inds = sympy.Matrix(tomato_12.drop(['target', 'date'], axis = 1)).rref()
#
# tomato_12.drop(['target', 'date'], axis = 1).iloc[:, [*inds]]\
#     .to_csv('Dataframes/X for Poisson-based regressions.csv', index = False)

In [4]:
set(tomato_12.drop(['target', 'date'], axis = 1).columns) -set(pd.read_csv('Dataframes/X for Poisson-based regressions.csv'))

set()

In [5]:
# GETTING RID OF HIGHLY CORRELATED COLUMNS

# https://www.projectpro.io/recipes/drop-out-highly-correlated-features-in-python

X_preprocessing = pd.read_csv('Dataframes/X for Poisson-based regressions.csv')

cor_matrix = X_preprocessing.corr().abs()
upper_tri = cor_matrix.where(np.triu(np.ones(cor_matrix.shape), k = 1).astype(np.bool))
to_drop = [column for column in upper_tri.columns if any(upper_tri[column] > 0.95)]

X = X_preprocessing.drop(to_drop, axis = 1)

In [6]:
# SETTING X, y

date = tomato_12['date']
y = tomato_12['target']

def timesplit(X, y, train_size = 0.8):
    n = len(y)
    n_train = int(n * train_size)
    X_train, X_test = X[:n_train], X[n_train:]
    y_train, y_test = y[:n_train], y[n_train:]
    return X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test = timesplit(X, y)

In [7]:
print('variance = ' + str(y_train.var()))
print('mean = ' + str(y_train.mean()))

variance = 6.814139319060391
mean = 1.4956772334293948


In [9]:
import statsmodels.api as sm

plus = 1
range_col_names = [[d for d in range(r, r + plus)] for r in range(0, len(X.columns) - plus, plus)]
range_col_names.append([r for r in range([*range(0, len(X.columns) - 1, plus)][-1], len(X.columns))])

X_in_for = pd.DataFrame()

for col_names in range_col_names:
    X_in_for = pd.concat([X_in_for, X.iloc[:, col_names]], axis = 1)
    X_train, X_test, y_train, y_test = timesplit(X_in_for, y)

    for p_value in [0.02, 0.01, 0.005, 0.002, 0.001]:
        Generalized_Poisson_reg = sm.GeneralizedPoisson(y_train, X_train, p = 1).fit()

        signif_var = pd.DataFrame([x for x in Generalized_Poisson_reg.summary().tables[1].data[1:] if float(x[4]) < p_value],
                                  columns = Generalized_Poisson_reg.summary().tables[1].data[0]).iloc[:-1, :]

        if len([*signif_var.iloc[:, 0]]) > 0:
            X_in_for = X_in_for.loc[:, [*signif_var.iloc[:, 0]]]
            X_train, X_test, y_train, y_test = timesplit(X_in_for, y)

        # print(str(p_value) +' : ' + Generalized_Poisson_reg.summary().tables[0].data[4][2:][0] +
        #       Generalized_Poisson_reg.summary().tables[0].data[4][2:][1] + '; len of X: ' + str(len(X_train.columns)))

print(Generalized_Poisson_reg.summary())
GP_train_pred = Generalized_Poisson_reg.predict(X_train)
GP_test_pred = Generalized_Poisson_reg.predict(X_test)

Optimization terminated successfully.
         Current function value: 1.689750
         Iterations: 7
         Function evaluations: 8
         Gradient evaluations: 8
0.02 :   Log-Likelihood:     -2931.7; len of X: 1
Optimization terminated successfully.
         Current function value: 1.689750
         Iterations: 7
         Function evaluations: 8
         Gradient evaluations: 8
0.01 :   Log-Likelihood:     -2931.7; len of X: 1
Optimization terminated successfully.
         Current function value: 1.689750
         Iterations: 7
         Function evaluations: 8
         Gradient evaluations: 8
0.005 :   Log-Likelihood:     -2931.7; len of X: 1
Optimization terminated successfully.
         Current function value: 1.689750
         Iterations: 7
         Function evaluations: 8
         Gradient evaluations: 8
0.002 :   Log-Likelihood:     -2931.7; len of X: 1
Optimization terminated successfully.
         Current function value: 1.689750
         Iterations: 7
         Function e

C:\Users\Владислав\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\base\model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


Optimization terminated successfully.
         Current function value: 1.346709
         Iterations: 32
         Function evaluations: 39
         Gradient evaluations: 39
0.01 :   Log-Likelihood:     -2336.5; len of X: 7
Optimization terminated successfully.
         Current function value: 1.346709
         Iterations: 32
         Function evaluations: 39
         Gradient evaluations: 39
0.005 :   Log-Likelihood:     -2336.5; len of X: 7
Optimization terminated successfully.
         Current function value: 1.346709
         Iterations: 32
         Function evaluations: 39
         Gradient evaluations: 39
0.002 :   Log-Likelihood:     -2336.5; len of X: 7
Optimization terminated successfully.
         Current function value: 1.346709
         Iterations: 32
         Function evaluations: 39
         Gradient evaluations: 39
0.001 :   Log-Likelihood:     -2336.5; len of X: 7
Optimization terminated successfully.
         Current function value: 1.344930
         Iterations: 33
     

C:\Users\Владислав\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\base\model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


Optimization terminated successfully.
         Current function value: 1.346709
         Iterations: 32
         Function evaluations: 39
         Gradient evaluations: 39
0.002 :   Log-Likelihood:     -2336.5; len of X: 7
Optimization terminated successfully.
         Current function value: 1.346709
         Iterations: 32
         Function evaluations: 39
         Gradient evaluations: 39
0.001 :   Log-Likelihood:     -2336.5; len of X: 7
Optimization terminated successfully.
         Current function value: 1.345980
         Iterations: 32
         Function evaluations: 39
         Gradient evaluations: 39
0.02 :   Log-Likelihood:     -2335.3; len of X: 7
Optimization terminated successfully.
         Current function value: 1.346709
         Iterations: 32
         Function evaluations: 39
         Gradient evaluations: 39
0.01 :   Log-Likelihood:     -2336.5; len of X: 7
Optimization terminated successfully.
         Current function value: 1.346709
         Iterations: 32
      

...

C:\Users\Владислав\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\base\model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
C:\Users\Владислав\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\base\model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


0.01 :   Log-Likelihood:         nan; len of X: 12
         Current function value: nan
         Iterations: 0
         Function evaluations: 16
         Gradient evaluations: 16
0.005 :   Log-Likelihood:         nan; len of X: 12
         Current function value: nan
         Iterations: 0
         Function evaluations: 16
         Gradient evaluations: 16
0.002 :   Log-Likelihood:         nan; len of X: 12


C:\Users\Владислав\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\base\model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
C:\Users\Владислав\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\base\model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


         Current function value: nan
         Iterations: 0
         Function evaluations: 16
         Gradient evaluations: 16
0.001 :   Log-Likelihood:         nan; len of X: 12
                    GeneralizedPoisson Regression Results                     
Dep. Variable:                 target   No. Observations:                 1735
Model:             GeneralizedPoisson   Df Residuals:                     1723
Method:                           MLE   Df Model:                           11
Date:                Thu, 21 Apr 2022   Pseudo R-squ.:                     nan
Time:                        17:24:45   Log-Likelihood:                    nan
converged:                      False   LL-Null:                       -2884.5
Covariance Type:            nonrobust   LLR p-value:                       nan
                           coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------
av_price_2

C:\Users\Владислав\AppData\Local\Programs\Python\Python310\lib\site-packages\statsmodels\base\model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


In [ ]:
# import statsmodels.api as sm
#
# for p_value in [i / 1000 for i in range(50, 0, -1)] + [0.001] * 5 + [0.0005] * 5:
#     Generalized_Poisson_reg = sm.GeneralizedPoisson(y_train, X_train, p = 1).fit()
#
#     signif_var = pd.DataFrame([x for x in Generalized_Poisson_reg.summary().tables[1].data[1:] if float(x[4]) < p_value],
#                               columns = Generalized_Poisson_reg.summary().tables[1].data[0]).iloc[:-1, :]
#
#     if len([*signif_var.iloc[:, 0]]) > 0:
#         X_train, X_test, y_train, y_test = timesplit(X.loc[:, [*signif_var.iloc[:, 0]]], y)
#
#     print(str(p_value) +' : ' + Generalized_Poisson_reg.summary().tables[0].data[4][2:][0] +
#           Generalized_Poisson_reg.summary().tables[0].data[4][2:][1] + '; len of X: ' + str(len(X_train.columns)))
#
# print(Generalized_Poisson_reg.summary())
# GP_train_pred = Generalized_Poisson_reg.predict(X_train)
# GP_test_pred = Generalized_Poisson_reg.predict(X_test)

In [ ]:
from sklearn import metrics

print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, GP_test_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, GP_test_pred))

In [38]:
a

In [39]:
# GENERALIZED POISSON test

import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scatter(x = tomato_12.loc[X_test.index, 'date'],
                         y = y_test,
                         mode = 'lines+markers',
                         name = 'Actual counts',
                         marker = dict(color = '#00A383', size = 3.5),
                         line = dict(color = '#00A383', width = 1.5)))

fig.add_trace(go.Scatter(x = tomato_12.loc[X_test.index, 'date'],
                         # y = [*map(round, GBDT_test_pred)],
                         y = GP_test_pred,
                         mode = 'lines',
                         name = 'Predicted counts',
                         line = dict(color = '#F53D65', width = 2.5)
                         ))

fig.add_trace(go.Scatter(x = tomato_12.loc[X_test.index, 'date'],
                         y = tomato_discount[tomato_discount['date']\
                         .isin(tomato_12.loc[X_test.index, 'date'])]\
                         .loc[:, 'is_discount_201676'] * 50,
                         mode = 'lines',
                         name = 'Presence of discount',
                         line = dict(color = '#B1F100', width = 2.5, dash = 'dot')  # 'dash'
                         ))

fig.show()